### Постановка задачи

Требуется на основании демографических данных о клиентах банка, истории их платежей и состоянии счета, предсказать невыполнение их обязательств по кредитной карте в следующем месяце.

**Описание датасета**

* **Home Ownership** - Домовладение (Home Mortgage, Rent, Own Home, Have Mortgage)
* **Annual Income** - Годовой доход
* **Years in current job** - Количество лет на текущем месте работы
* **Tax Liens** - Налоговые обременения
* **Number of Open Accounts** - Количество открытых счетов
* **Years of Credit History** - Количество лет кредитной истории
* **Maximum Open Credit** - Наибольший открытый кредит
* **Number of Credit Problems** - Количество проблем с кредитом
* **Months since last delinquent** - Количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротство
* **Purpose** - Цель кредита (debt consolidation, other, home improvements, business loan, buy a car, medical bills, major purchase, take a trip, buy house, small business, wedding, moving, educational expenses, vacation, renewable energy)
* **Term** - Срок кредита (Long Term, Short Term)
* **Current Loan Amount** - Текущая сумма кредита
* **Current Credit Balance** - Текущий кредитный баланс
* **Monthly Debt** - Ежемесячный долг
* **Credit Score** - Счет кредита
* **Credit Default** -  факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import shapiro
from scipy.stats import probplot
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats import chi2_contingency
from statsmodels.stats.weightstats import zconfint

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
# TEST_DATASET_PATH = 'course_project_test.csv'
# df = pd.read_csv(TEST_DATASET_PATH)
# df.head()

In [9]:
DATASET_PATH = 'course_project_train.csv'

# output
PREP_DATASET_PATH = 'course_project_train_prep.csv'

In [10]:
df = pd.read_csv(DATASET_PATH)
df.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [11]:
df.shape

(7500, 17)

In [12]:
df.iloc[0]

Home Ownership                            Own Home
Annual Income                               482087
Years in current job                           NaN
Tax Liens                                        0
Number of Open Accounts                         11
Years of Credit History                       26.3
Maximum Open Credit                         685960
Number of Credit Problems                        1
Months since last delinquent                   NaN
Bankruptcies                                     1
Purpose                         debt consolidation
Term                                    Short Term
Current Loan Amount                          1e+08
Current Credit Balance                       47386
Monthly Debt                                  7914
Credit Score                                   749
Credit Default                                   0
Name: 0, dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 17 columns):
Home Ownership                  7500 non-null object
Annual Income                   5943 non-null float64
Years in current job            7129 non-null object
Tax Liens                       7500 non-null float64
Number of Open Accounts         7500 non-null float64
Years of Credit History         7500 non-null float64
Maximum Open Credit             7500 non-null float64
Number of Credit Problems       7500 non-null float64
Months since last delinquent    3419 non-null float64
Bankruptcies                    7486 non-null float64
Purpose                         7500 non-null object
Term                            7500 non-null object
Current Loan Amount             7500 non-null float64
Current Credit Balance          7500 non-null float64
Monthly Debt                    7500 non-null float64
Credit Score                    5943 non-null float64
Credit Default                  7

Переведём некоторые объекты в str вид

In [14]:
for colname in ['Tax Liens', 'Number of Credit Problems','Number of Open Accounts', 'Bankruptcies']:
    df[colname] = df[colname].astype(str)

In [15]:
df.dtypes

Home Ownership                   object
Annual Income                   float64
Years in current job             object
Tax Liens                        object
Number of Open Accounts          object
Years of Credit History         float64
Maximum Open Credit             float64
Number of Credit Problems        object
Months since last delinquent    float64
Bankruptcies                     object
Purpose                          object
Term                             object
Current Loan Amount             float64
Current Credit Balance          float64
Monthly Debt                    float64
Credit Score                    float64
Credit Default                    int64
dtype: object

Проверим просрочку платежа (1=да, 0=нет)

In [16]:
df['Credit Default'].value_counts()

0    5387
1    2113
Name: Credit Default, dtype: int64

Почти у 1/3 от всех клиентов есть просрочка платежа

**Обзор количественных признаков**

In [17]:
df.describe()

,Annual Income,Years of Credit History,Maximum Open Credit,Months since last delinquent,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
count,5.943000e+03,7500.000000,7.500000e+03,3419.000000,7.500000e+03,7.500000e+03,7500.000000,5943.000000,7500.000000
mean,1.366392e+06,18.317467,9.451537e+05,34.692600,1.187318e+07,2.898332e+05,18314.454133,1151.087498,0.281733
std,8.453392e+05,7.041946,1.602622e+07,21.688806,3.192612e+07,3.178714e+05,11926.764673,1604.451418,0.449874
min,1.645970e+05,4.000000,0.000000e+00,0.000000,1.124200e+04,0.000000e+00,0.000000,585.000000,0.000000
25%,8.443410e+05,13.500000,2.792295e+05,16.000000,1.801690e+05,1.142565e+05,10067.500000,711.000000,0.000000
50%,1.168386e+06,17.000000,4.781590e+05,32.000000,3.095730e+05,2.093230e+05,16076.500000,731.000000,0.000000
75%,1.640137e+06,21.800000,7.935015e+05,50.000000,5.198820e+05,3.604062e+05,23818.000000,743.000000,1.000000
max,1.014934e+07,57.700000,1.304726e+09,118.000000,1.000000e+08,6.506797e+06,136679.000000,7510.000000,1.000000


Все значения в разумных приделах

**Обзор номинативных признаков**

In [18]:
for cat_colname in df.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

Home Ownership

Home Mortgage    3637
Rent             3204
Own Home          647
Have Mortgage      12
Name: Home Ownership, dtype: int64
****************************************************************************************************

Years in current job

10+ years    2332
2 years       705
3 years       620
< 1 year      563
5 years       516
1 year        504
4 years       469
6 years       426
7 years       396
8 years       339
9 years       259
Name: Years in current job, dtype: int64
****************************************************************************************************

Tax Liens

0.0    7366
1.0      83
2.0      30
3.0      10
4.0       6
6.0       2
5.0       2
7.0       1
Name: Tax Liens, dtype: int64
****************************************************************************************************

Number of Open Accounts

9.0     728
11.0    692
10.0    677
8.0     638
7.0     613
12.0    562
6.0     504
13.0    465
14.0    420
5.0     325
15.0    313


### Обработка пропусков

In [19]:
len(df) - df.count()

Home Ownership                     0
Annual Income                   1557
Years in current job             371
Tax Liens                          0
Number of Open Accounts            0
Years of Credit History            0
Maximum Open Credit                0
Number of Credit Problems          0
Months since last delinquent    4081
Bankruptcies                       0
Purpose                            0
Term                               0
Current Loan Amount                0
Current Credit Balance             0
Monthly Debt                       0
Credit Score                    1557
Credit Default                     0
dtype: int64

In [20]:
for colname in ['Number of Credit Problems', 'Bankruptcies']:
    df[colname] = df[colname].astype(str)

In [21]:
df = df.fillna(0)

In [22]:
df.loc[df['Annual Income'] == 'nan', 'Annual Income'] = 0

In [23]:
df.loc[df['Annual Income'] == 0, 'Annual Income'] = df['Annual Income'].mean()

In [24]:
df.loc[df['Years in current job'] == 0, 'Years in current job'] = df['Years in current job'].mode()[0]

In [25]:
df.loc[df['Months since last delinquent'] == 0, 'Months since last delinquent'] = df['Months since last delinquent'].mean()

In [26]:
df.loc[df['Credit Score'] == 0, 'Credit Score'] = df['Credit Score'].mean()

Заменили пропуски в 'Annual Income', 'Months since last delinquent', 'Credit Score' средими значениями, а в 'Years in current job' самым частым значением

### Обработка выбросов

**Tax Liens**

С большой вероятностью 5, 6, 7 - выбросы

In [27]:
df.loc[df['Tax Liens'] == '5.0', 'Tax Liens'] = df['Tax Liens'].mode()[0]
df.loc[df['Tax Liens'] == '6.0', 'Tax Liens'] = df['Tax Liens'].mode()[0]
df.loc[df['Tax Liens'] == '7.0', 'Tax Liens'] = df['Tax Liens'].mode()[0]

In [28]:
df['Tax Liens'].value_counts()

0.0    7371
1.0      83
2.0      30
3.0      10
4.0       6
Name: Tax Liens, dtype: int64

**Number of Credit Problems**

С большой вероятностью 6, 7 - выбросы

In [29]:
df.loc[df['Number of Credit Problems'] == '7.0', 'Number of Credit Problems'] = df['Number of Credit Problems'].mode()[0]
df.loc[df['Number of Credit Problems'] == '6.0', 'Number of Credit Problems'] = df['Number of Credit Problems'].mode()[0]

In [30]:
df['Number of Credit Problems'].value_counts()

0.0    6474
1.0     882
2.0      93
3.0      35
4.0       9
5.0       7
Name: Number of Credit Problems, dtype: int64

**Bankruptcies**

С большой вероятностью 4 - выбросы

In [31]:
df.loc[df['Bankruptcies'] == '4.0', 'Bankruptcies'] = df['Bankruptcies'].mode()[0]

In [32]:
df['Bankruptcies'].value_counts()

0.0    6662
1.0     786
2.0      31
nan      14
3.0       7
Name: Bankruptcies, dtype: int64

### Построение новых признаков

**Id**

In [33]:
df['ID'] = df.index.tolist()

In [34]:
df.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default,ID
0,Own Home,482087.0,10+ years,0.0,11.0,26.3,685960.0,1.0,15.8152,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,15.8152,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,15.8152,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0,2
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,15.8152,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0,3
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,15.8152,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0,4


### Сохранение обучающего датасета

In [35]:
df.to_csv(PREP_DATASET_PATH, index=False, encoding='utf-8')